In [17]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# YouTube Video Analysis with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fvideo-analysis%2Fyoutube_video_analysis.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/video-analysis/youtube_video_analysis.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| | |
|-|-|
| Author(s) | [Alok Pattani](https://github.com/alokpattani/) |

## Overview

In this notebook, you'll explore how to do direct analysis of publicly available [YouTube](https://www.youtube.com/) videos with Gemini.

You will complete the following tasks:

- Summarizing a single YouTube video using Gemini 2.0 Flash
- Extracting a specific set of structured outputs from a longer YouTube video using Gemini 2.0 Pro and controlled generation
- Creating insights from analyzing multiple YouTube videos together using asynchronous generation with Gemini

## Get started

### Install Google Gen AI SDK and other required packages


In [1]:
%pip install --upgrade --quiet google-genai itables

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [19]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [12]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = "gentle-current-461511-j5"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "frencheaze":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Set up libraries, options, and models

### Import libraries

In [4]:
import json

from IPython.display import HTML, Markdown, display
from google.genai.types import GenerateContentConfig, Part
from itables import show
import itables.options as itable_opts
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_random_exponential

### Configure some notebook options

In [5]:
# Configure some options related to interactive tables
itable_opts.maxBytes = 1e9
itable_opts.maxColumns = 50

itable_opts.order = []
itable_opts.column_filters = "header"

### Create a helper function

In [6]:
def display_youtube_video(url: str) -> None:
    youtube_video_embed_url = url.replace("/watch?v=", "/embed/")

    # Create HTML code to directly embed video
    youtube_video_embed_html_code = f"""
    <iframe width="560" height="315" src="{youtube_video_embed_url}"
    title="YouTube video player" frameborder="0" allow="accelerometer; autoplay;
    clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen>
    </iframe>
    """

    # Display embedded YouTube video
    display(HTML(youtube_video_embed_html_code))

### Load models

In [7]:
# Set Gemini Flash and Pro models to be used in this notebook
GEMINI_FLASH_MODEL_ID = "gemini-2.0-flash-001"
GEMINI_PRO_MODEL_ID = "gemini-2.0-flash"

## Summarize a YouTube video

Provide a link to a public YouTube video that you'd like to summarize. Ensure that the video is less than an hour long (if using Gemini 2.0, as is shown below; can try up to a 2-hour video with Gemini 2.0) to make sure it fits in the context window.

The default content to be summarized is [this 6.5-minute video showing how Major League Baseball (MLB) analyzes data using Google Cloud](https://www.youtube.com/watch?v=O_W_VGUeHVI).

In [8]:
# Provide link to a public YouTube video to summarize
YOUTUBE_VIDEO_URL = (
    "https://www.youtube.com/watch?v=O_W_VGUeHVI"  # @param {type:"string"}
)

display_youtube_video(YOUTUBE_VIDEO_URL)

In [13]:
# Call Gemini API with prompt to summarize video
video_summary_prompt = "Give a detailed summary of this video."

video_summary_response = client.models.generate_content(
    model=GEMINI_FLASH_MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri=YOUTUBE_VIDEO_URL,
            mime_type="video/webm",
        ),
        video_summary_prompt,
    ],
)

# Display results
display(Markdown(video_summary_response.text))

Here’s a detailed summary of the video:

0:00​–0:07 The speaker introduces the topic of the video: MLB’s partnership with Google Cloud to answer deep questions with data.
0:07-0:24 There is a montage that shows black and white footage of baseball throughout the decades.
0:24–0:38 The video transitions to show the Google Cloud being the “Official Cloud Technology of MLB."
0:38–0:54 The video transitions to the data analysis used to answer burning questions that fans want to be answered.
0:54–1:04 Josh Frost, Vice President, Product Management at MLB is introduced, and discusses how 25 million unique data points are captured per game.
1:04–1:14 There is a montage that shows amazing saves and celebrations by players.
1:14–1:25 Alok Pattani, Data Science Developer Advocate at Google, is introduced.
1:25–1:39 Pattani discusses how the data in MLB’s Film Room is tracked many times a second, with instrumentation and technology helping to process that data.
1:39–2:03 The video transitions to Rob Engel, Senior Director Software Engineering at MLB. He talks about using Anthos, a deployed Kubernetes inside the ballpark, which sends the data to the Google Cloud Kubernetes engine. This is then used to create stats using data from the MLB.
2:03–2:23 Back with Priyanka Vergadia and Rob Engel, as she asks how it all works, and how the data is collected and processed.
2:23–2:31 Engel explains how the cameras are tracking everything about that pitch in real-time, at 30 frames per second.
2:31–2:45 The video shows how those cameras have 60 data points about every single pitch. Data processors collect the data from Hawkeye and hold it in their data center.
2:45–2:55 He says that data can then be queried, and that they need a highly scalable technology such as BigTable.
2:55–3:01 Engel notes how BigTable is awesome to use for post-tracking.
3:01–3:15 He then explains how they have a project called “Field Vision,” which creates a constant stream of pose-tracking data that is visible to the fans on their computer.
3:15–3:30 Veragadia questions further processing and making it available for APIs.
3:30–3:47 The video displays a graphic of what the process flow is: 
Anthos In-ballpark Tools > Kubernetes Engine Gameday Engine > BigTable Cloud Storage > DataFlow Cloud Composer > Big Query MLB Stats API.
3:47–3:57 Engel explains how this is all is a home field advantage.
3:57–4:27 John Krazit, Director, Baseball Systems at Arizona Diamondbacks, is introduced.
4:27–4:55 She asks John how his team utilizes data.
4:55–5:06 The scene reverts to Veragadia, who asks how analysts can use data from the cloud, so that they are able to translate into something the average fan cares about.
5:06–5:28 Sarah Langs, Reporter/Researcher at MLB Advanced Media, is introduced, saying that it depends on what information she needs.
5:28–5:57 Langs discusses how she would use Baseball Savant, and then talks about how Statcast allows measurement of everything, in a way that will speak to the average fan.
5:57-6:30 The video concludes with how sports data analytics is a powerful tool, thanking the viewer for joining her on this journey.

## Extract structured output from a YouTube video

Next, we'll show how to extract structured outputs using [controlled generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output), in this case from a video that covers multiple topics.

We're going to see how Gemini's industry-leading 2 million token context window can help analyze [the full opening keynote](https://www.youtube.com/watch?v=V6DJYGn2SFk) from our Next conference back in April - all 1 hour and 41 minutes of it!

In [10]:
# Link to full Cloud Next '24 Opening Keynote video
# cloud_next_keynote_video_url = "https://www.youtube.com/watch?v=V6DJYGn2SFk"

# Uncomment line below to replace with 14-min keynote summary video instead (faster)
cloud_next_keynote_video_url = "https://www.youtube.com/watch?v=M-CzbTUVykg"

display_youtube_video(cloud_next_keynote_video_url)

Below is a prompt to extract the biggest product announcements that were made during this keynote. We use the response schema to show that we want valid JSON output in a particular form, including a constraint specifying that the "product status" field should be either GA, Preview, or Coming Soon.

The following cell may take several minutes to run, as Gemini 2.0 Pro is analyzing all 101 minutes of the video and audio to produce comprehensive results.

In [14]:
# Set up pieces (prompt, response schema, config) and run video extraction

video_extraction_prompt = (
    "Provide a summary of the biggest product announcements "
    "that were made in this Google Cloud Next keynote video including:\n"
    "  - name\n"
    '  - product status: "GA" (Generally Available), "Preview", or "Coming Soon"\n'
    "  - key quote from the presenter about the product, 20 words or fewer per product\n\n"
    "Make sure to look through and listen to the whole video, start to finish, to find "
    "the top product announcements. Only reference information in the video itself in "
    "your response."
)

video_extraction_response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "name": {"type": "STRING"},
            "product_status": {
                "type": "STRING",
                "enum": ["GA", "Preview", "Coming Soon"],
            },
            "quote_from_presenter": {"type": "STRING"},
        },
    },
}

video_extraction_json_generation_config = GenerateContentConfig(
    temperature=0.0,
    max_output_tokens=8192,
    response_mime_type="application/json",
    response_schema=video_extraction_response_schema,
)

video_extraction_response = client.models.generate_content(
    model=GEMINI_PRO_MODEL_ID,
    contents=[
        video_extraction_prompt,
        Part.from_uri(
            file_uri=cloud_next_keynote_video_url,
            mime_type="video/webm",
        ),
    ],
    config=video_extraction_json_generation_config,
)

print(video_extraction_response.text)

[
  {
    "name": "Gemini 1.5 Pro",
    "product_status": "Preview",
    "quote_from_presenter": "Bringing the world's largest context window to developers."
  },
  {
    "name": "A3 Mega VMs",
    "product_status": "GA",
    "quote_from_presenter": "2x the network bandwidth vs. A3."
  },
  {
    "name": "NVIDIA GB200 NVL72",
    "product_status": "Coming Soon",
    "quote_from_presenter": "Coming in early 2025."
  },
  {
    "name": "AI Optimized Storage",
    "product_status": "Preview",
    "quote_from_presenter": "Accelerating inference with Hyperdisk ML."
  },
  {
    "name": "Google Axion Processor",
    "product_status": "Preview",
    "quote_from_presenter": "Our first custom Arm-based CPU designed for the data center."
  },
  {
    "name": "Vertex AI Agent Builder",
    "product_status": "Preview",
    "quote_from_presenter": "You can now create customer agents that are amazingly powerful in just three key steps."
  },
  {
    "name": "AI Meetings and Messaging Add-on",
    "p

In [15]:
# Convert structured output from response to data frame for display and/or further analysis
video_extraction_response_df = pd.DataFrame(video_extraction_response.parsed)

show(video_extraction_response_df)

/usr/local/lib/python3.11/dist-packages/itables/typing.py:196: SyntaxWarning: maxBytes=1000000000.0 does not match NotRequired[Union[int, str]]: value of key 'maxBytes' of dict did not match any element in the union:
  int: is not an instance of int
  str: is not an instance of str. You can silence this warning by setting `itables.options.warn_on_undocumented_option=False`. If you believe ITableOptions should be updated, please make a PR or open an issue at https://github.com/mwouts/itables
  warnings.warn(


Loading ITables v2.4.0 from the internet... (need help?)


## Creating insights from analyzing multiple YouTube videos together

### Google "Year in Search" videos
Now, consider expanding the problem to a more common enterprise use case: extracting information from _multiple_ YouTube videos at once.

This time, we'll use [Google's "Year in Search" videos](https://about.google/intl/ALL_us/stories/year-in-search/), which summarize the questions, people, and moments that captured the world's attention in each year. As of fall 2024, there are 14 of these videos, each 2-4 minutes in length, from [2010](https://www.youtube.com/watch?v=F0QXB5pw2qE) through [2023](https://www.youtube.com/watch?v=3KtWfp0UopM).

We start by reading in a CSV file that has links to all the videos.

In [16]:
# Read in table of Year in Search video links from public CSV file
GOOGLE_YEAR_IN_SEARCH_VIDEO_LINKS_CSV_GCS_URI = (
    "gs://github-repo/video/google_year_in_search_video_links.csv"
)

year_in_search_yt_links = pd.read_csv(GOOGLE_YEAR_IN_SEARCH_VIDEO_LINKS_CSV_GCS_URI)

year_in_search_yt_links

,year,yt_link
0,2023,https://www.youtube.com/watch?v=3KtWfp0UopM
1,2022,https://www.youtube.com/watch?v=4WXs3sKu41I
2,2021,https://www.youtube.com/watch?v=EqboAI-Vk-U
3,2020,https://www.youtube.com/watch?v=rokGy0huYEA
4,2019,https://www.youtube.com/watch?v=ZRCdORJiUgU
5,2018,https://www.youtube.com/watch?v=6aFdEhEZQjE
6,2017,https://www.youtube.com/watch?v=vI4LHl4yFuo
7,2016,https://www.youtube.com/watch?v=KIViy7L_lo8
8,2015,https://www.youtube.com/watch?v=q7o7R5BgWDY
9,2014,https://www.youtube.com/watch?v=DVwHCGAr_OE


### Set up for analyzing multiple video files

Let's say we are a sports agency who wants to see which athletes or teams appear most often in these videos as a measure of cultural relevance. Instead of watching and manually counting, we can use Gemini's multimodal capabilities and world knowledge to extract each appearance of an athlete or team into a structured output that we can use for further analysis.

The system instructions, prompt, and response schema that will apply to all 14 videos are each created in the cell below.

In [17]:
# Set up pieces (prompt, response schema, config) for Google Year in Search videos
multiple_video_extraction_system_instruction_text = (
    "You are a video analyst that "
    "carefully looks through all frames of provided videos, extracting out the "
    "pieces necessary to respond to user prompts."
)

multiple_video_extraction_prompt = (
    "Which sports athletes or teams are mentioned or "
    "shown in this video? Please look through each frame carefully, and respond "
    "with a complete list that includes the athlete or team's name (1 row per "
    "athlete or team), whether they are an athlete or team, the sport they play, "
    "and the timestamp into the video at which they appear (in mm:ss format, "
    "do not give extra precision) for each one."
)

multiple_video_extraction_response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "name": {"type": "STRING"},
            "athlete_or_team": {"type": "STRING", "enum": ["athlete", "team"]},
            "sport": {"type": "STRING"},
            "video_timestamp": {"type": "STRING"},
        },
    },
}

multiple_video_extraction_json_generation_config = GenerateContentConfig(
    temperature=0.0,
    max_output_tokens=8192,
    response_mime_type="application/json",
    response_schema=multiple_video_extraction_response_schema,
)

Next, we'll set up to run each of these prompt/video pairs through the Gemini API _asynchronously_. This allows us to send all the requests to Gemini at once, then wait for all the answers to come back - a more efficient process than sending them synchronously (one-by-one). See more details in [this Google Cloud Community Medium blog post](https://medium.com/google-cloud/how-to-prompt-gemini-asynchronously-using-python-on-google-cloud-986ca45d9f1b).


In [18]:
# Function for asynchronous generation


@retry(wait=wait_random_exponential(multiplier=1, max=120), stop=stop_after_attempt(2))
async def async_generate(prompt, yt_link):
    try:
        response = await client.aio.models.generate_content(
            model=GEMINI_PRO_MODEL_ID,
            contents=[prompt, Part.from_uri(file_uri=yt_link, mime_type="video/webm")],
            config=multiple_video_extraction_json_generation_config,
        )

        return response.to_json_dict()
    except Exception as e:
        print("Something failed, retrying")
        print(e)
        with retry.stop_after_attempt(2) as retry_state:
            if retry_state.attempt > 2:
                return None
        raise  # Re-raise the exception for tenacity to handle

### Run asynchronous Gemini calls to do video extraction

In [19]:
# Perform asynchronous calls across all videos, gather responses
import asyncio

start_time = asyncio.get_event_loop().time()

get_responses = [
    async_generate(multiple_video_extraction_prompt, yt_link)
    for yt_link in year_in_search_yt_links["yt_link"]
]

multiple_video_extraction_responses = await asyncio.gather(*get_responses)

end_time = asyncio.get_event_loop().time()

elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 21.66 seconds


### Extract and analyze video results across years

Once we have the results from Gemini, we can process them and get table of every athlete or team appearance across all 14 "Year in Search" videos.

In [20]:
# Add structured outputs by year back to original table, show full extraction results
year_in_search_responses = year_in_search_yt_links.copy()

year_in_search_responses["gemini_response"] = [
    json.dumps(response) for response in multiple_video_extraction_responses
]


def extract_result_df_from_gemini_response(year, gemini_response):
    extract_response_text = json.loads(gemini_response)["candidates"][0]["content"][
        "parts"
    ][0]["text"]

    extract_result_df = pd.DataFrame(json.loads(extract_response_text))

    extract_result_df["year"] = year

    return extract_result_df


year_in_search_responses["extract_result_df"] = year_in_search_responses.apply(
    lambda row: extract_result_df_from_gemini_response(
        row["year"], row["gemini_response"]
    ),
    axis=1,
)

all_year_in_search_extractions = pd.concat(
    year_in_search_responses["extract_result_df"].tolist(), ignore_index=True
)[["year", "name", "athlete_or_team", "sport", "video_timestamp"]]

show(all_year_in_search_extractions)

/usr/local/lib/python3.11/dist-packages/itables/typing.py:196: SyntaxWarning: maxBytes=1000000000.0 does not match NotRequired[Union[int, str]]: value of key 'maxBytes' of dict did not match any element in the union:
  int: is not an instance of int
  str: is not an instance of str. You can silence this warning by setting `itables.options.warn_on_undocumented_option=False`. If you believe ITableOptions should be updated, please make a PR or open an issue at https://github.com/mwouts/itables
  warnings.warn(


Loading ITables v2.4.0 from the internet... (need help?)


Finally, we can count the number of years in which each athlete or team appeared in these videos, and return results for those who appeared more than once.

In [21]:
# Analyze results to show athletes/teams showing up most often in Year in Search videos
multiple_year_in_search_app = (
    all_year_in_search_extractions.assign(
        # Convert 'name' to uppercase to handle e.g. "LeBron" vs "Lebron"
        name=all_year_in_search_extractions["name"].str.upper(),
        # Convert 'athlete_or_team' to lowercase for consistency
        athlete_or_team=all_year_in_search_extractions["athlete_or_team"].str.lower(),
    )
    .groupby(["name", "athlete_or_team"])
    .apply(
        lambda x: pd.Series(
            {
                # Aggregate 'sport' across type and name (handling different cases)
                "sport": ", ".join(sorted(x["sport"].str.lower().unique())),
                # Count # of diff years in which each athlete/team appears in video
                "num_years": x["year"].nunique(),
            }
        )
    )
    .reset_index()
    .
    # Filter to only those appearing multiple times
    query("num_years >= 2")
    .sort_values(["num_years", "name"], ascending=[False, True])
    .reset_index(drop=True)
)

# Display results
display(Markdown("<b>Athletes/Teams Appearing in Multiple Year in Search Videos<b>"))
display(multiple_year_in_search_app)

<ipython-input-21-d810eb97520f>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


<b>Athletes/Teams Appearing in Multiple Year in Search Videos<b>

,name,athlete_or_team,sport,num_years
0,LEBRON JAMES,athlete,basketball,3
1,NAOMI OSAKA,athlete,tennis,3
2,SERENA WILLIAMS,athlete,tennis,3
3,SIMONE BILES,athlete,gymnastics,3
4,LEICESTER CITY,team,soccer,2
5,SPAIN,team,soccer,2
6,USAIN BOLT,athlete,track and field,2
